In [159]:
import pandas as pd
import numpy as np

pro = pd.read_csv(r"C:\Users\ABHI\Desktop\Sampled (1).csv")


In [160]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from skmultilearn.ensemble import RakelO

In [161]:
pro.head(3)

,Unnamed: 0,Unnamed: 0.1,airline_name,author,author_country,content,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,value_money_rating,Month,Year,recommended
0,5373,7371,ana-all-nippon-airways,S Bajumi,United States,New revised economy's seating arrangement on A...,Economy,6,2,4,4,4,2,5,2015,0
1,4342,5870,airasia-x,Paesen Yves,Belgium,KUL-MELMEL-KUL check in good and fast friendly...,Economy,10,5,5,4,0,5,11,2012,1
2,22935,35558,thomas-cook-airlines,P Phillips,United Kingdom,Just got back from Barbados on the (now) infam...,Premium Economy,9,4,5,4,3,5,11,2014,1


In [162]:
pro['content'] = pro['content'].str.replace('\d+', '')

In [163]:
import re
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [164]:
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

In [165]:
pro['content'] = pro['content'].astype(str)
pro['content'] = pro['content'].str.replace(r'[^\w\s]+', '')

In [166]:
from tqdm import tqdm
content = []
# tqdm is for printing the status bar
for sentance in tqdm(pro['content'].values):
    sentance.lower()
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', str(sent))
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e not in stopwords)
    content.append(sent.strip()) 

100%|█████████████████████████████████████| 2000/2000 [00:04<00:00, 488.53it/s]


In [167]:
lst = []
for i in pro['content']:
    lst.append(i.lower())

In [168]:
pro['content'] = lst

In [169]:
one_hot = pd.get_dummies(pro['cabin_flown'])
pro['Business Class'] = one_hot['Business Class']
pro['Economy'] = one_hot['Economy']
pro['First Class'] = one_hot['First Class']
pro['Premium Economy'] = one_hot['Premium Economy']

In [170]:
lst = []
for i in pro['overall_rating']:
    if int(i) > 5:
        lst.append(1)
    elif int(i) <= 5:
        lst.append(0)
    else:
        lst.append(None)

In [171]:
pro['overall_rating'] = lst

In [172]:
pro['overall_rating'].unique()

array([1, 0], dtype=int64)

In [173]:
from sklearn import preprocessing

x = pro['seat_comfort_rating']#returns a numpy array
x = x.values.reshape(-1, 1)
min_max_scaler = preprocessing.MinMaxScaler()
seat_comfort = min_max_scaler.fit_transform(x)
pro['seat_comfort_rating_1'] = seat_comfort

C:\Users\ABHI\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [174]:
x = pro['cabin_staff_rating']#returns a numpy array
x = x.values.reshape(-1, 1)
min_max_scaler = preprocessing.MinMaxScaler()
cabin_staff = min_max_scaler.fit_transform(x)
pro['cabin_staff_rating_1'] = cabin_staff

C:\Users\ABHI\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [175]:
x = pro['food_beverages_rating']#returns a numpy array
x = x.values.reshape(-1, 1)
min_max_scaler = preprocessing.MinMaxScaler()
food_beverages = min_max_scaler.fit_transform(x)
pro['food_beverages_rating_1'] = food_beverages

C:\Users\ABHI\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [176]:
x = pro['inflight_entertainment_rating']#returns a numpy array
x = x.values.reshape(-1, 1)
min_max_scaler = preprocessing.MinMaxScaler()
inflight_entertainment = min_max_scaler.fit_transform(x)
pro['inflight_entertainment_rating_11'] = inflight_entertainment

C:\Users\ABHI\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [177]:
pro.head(2)

,Unnamed: 0,Unnamed: 0.1,airline_name,author,author_country,content,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,...,Year,recommended,Business Class,Economy,First Class,Premium Economy,seat_comfort_rating_1,cabin_staff_rating_1,food_beverages_rating_1,inflight_entertainment_rating_11
0,5373,7371,ana-all-nippon-airways,S Bajumi,United States,new revised economys seating arrangement on an...,Economy,1,2,4,...,2015,0,0,1,0,0,0.4,0.8,0.8,0.8
1,4342,5870,airasia-x,Paesen Yves,Belgium,kulmelmelkul check in good and fast friendly s...,Economy,1,5,5,...,2012,1,0,1,0,0,1.0,1.0,0.8,0.0


In [178]:
lst = []
for i in pro['value_money_rating']:
    if int(i) > 3:
        lst.append(1)
    elif int(i) <= 3:
        lst.append(0)
    else:
        lst.append(None)

In [179]:
pro['value_money_rating_1'] = lst

In [180]:
pro.head(2)

,Unnamed: 0,Unnamed: 0.1,airline_name,author,author_country,content,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,...,recommended,Business Class,Economy,First Class,Premium Economy,seat_comfort_rating_1,cabin_staff_rating_1,food_beverages_rating_1,inflight_entertainment_rating_11,value_money_rating_1
0,5373,7371,ana-all-nippon-airways,S Bajumi,United States,new revised economys seating arrangement on an...,Economy,1,2,4,...,0,0,1,0,0,0.4,0.8,0.8,0.8,0
1,4342,5870,airasia-x,Paesen Yves,Belgium,kulmelmelkul check in good and fast friendly s...,Economy,1,5,5,...,1,0,1,0,0,1.0,1.0,0.8,0.0,1


In [23]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
model = KeyedVectors.load_word2vec_format(r"F:\Python\New folder\GoogleNews-vectors-negative300.bin", binary = True)

C:\Users\ABHI\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [181]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(pro, test_size=0.3, random_state = 50)

In [25]:
X_tr = train[['content', 'Business Class', 'Economy',  'First Class', 'Premium Economy', 'seat_comfort_rating_1', 'cabin_staff_rating_1', 'food_beverages_rating_1']]
y_tr = train[['overall_rating', 'value_money_rating_1', 'recommended']]
X_test = test[['content', 'Business Class', 'Economy',  'First Class', 'Premium Economy', 'seat_comfort_rating_1', 'cabin_staff_rating_1', 'food_beverages_rating_1']]
y_test = test[['overall_rating', 'value_money_rating_1', 'recommended']]


In [26]:
train.head(1)

,Unnamed: 0,Unnamed: 0.1,airline_name,author,author_country,content,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,...,recommended,Business Class,Economy,First Class,Premium Economy,seat_comfort_rating_1,cabin_staff_rating_1,food_beverages_rating_1,inflight_entertainment_rating_11,value_money_rating_1
1153,24755,37976,united-airlines,A Clark,United States,chicago to st louis roundtrip all flights were...,Economy,1,4,4,...,1,0,1,0,0,0.8,0.8,0.6,0.2,1


In [182]:
y_tr.nunique()

overall_rating          2
value_money_rating_1    2
recommended             2
dtype: int64

In [183]:
content = X_tr['content']

In [184]:
X_tr_preprocessed_summary111 = []
k = 0
li1 = []
for i in content:
    l=[]
    for j in i.split():
        try:
            l.append(model[str(j)])
            k = 1
        except:
            pass
    li1.append(np.mean(l,axis=0))

In [185]:
content = X_test['content']

In [186]:
X_tr_preprocessed_summary111 = []
li2 = []
for i in content:
    l=[]
    for j in i.split():
        try:
            l.append(model[str(j)])
            k = 1
        except:
            pass
    li2.append(np.mean(l,axis=0))

In [187]:
li1 = np.array(li1)
li1 = pd.DataFrame(li1)

In [188]:
#t = li1[0].apply(pd.Series)

In [189]:
li2 = np.array(li2)
li2 = pd.DataFrame(li2)

In [190]:
X_tr.head(2)

,content,Business Class,Economy,First Class,Premium Economy,seat_comfort_rating_1,cabin_staff_rating_1,food_beverages_rating_1
1153,chicago to st louis roundtrip all flights were...,0,1,0,0,0.8,0.8,0.6
1923,the inair part of my experience of this airlin...,0,1,0,0,0.4,0.6,0.6


In [192]:
X_tr_all = np.hstack((li1, X_tr[['Business Class', 'Economy', 'First Class', 'Premium Economy', 'seat_comfort_rating_1', 'cabin_staff_rating_1', 'food_beverages_rating_1']]))
print("the overall dimension of matrix is")
X_tr_all.shape

the overall dimension of matrix is


(1400, 307)

In [193]:
X_tr_all[0]

array([ 2.23260280e-02,  2.18972694e-02,  7.38841714e-03,  8.58884528e-02,
       -5.35451844e-02, -4.88579087e-02, -1.51220178e-02, -9.19295549e-02,
        9.79540274e-02,  6.24592490e-02, -9.72617511e-03, -1.11613065e-01,
        4.39264113e-03, -1.38047384e-02, -4.89709675e-02,  8.15616697e-02,
        7.03816265e-02,  9.59322378e-02,  2.44670957e-02, -2.09948439e-02,
       -5.01161329e-02,  8.96440446e-03,  1.11400066e-02, -1.12208622e-02,
        2.80197430e-02,  2.51457468e-02, -1.11487038e-01,  6.03588335e-02,
        4.44003306e-02,  9.42323264e-03, -2.02048607e-02, -2.85332836e-02,
       -3.26548330e-02, -1.69794150e-02,  1.56791508e-02, -3.09597570e-02,
       -3.00595351e-02,  2.36004149e-03,  1.59842558e-02,  5.02253026e-02,
        5.06241024e-02, -4.35295552e-02,  1.34230122e-01, -1.72909070e-02,
       -1.07886614e-02, -4.89612781e-02,  1.21114310e-02,  3.34390788e-03,
        4.00299914e-02, -2.62074359e-03,  2.35232841e-02,  4.18088846e-02,
        2.13070381e-02,  

In [195]:
X_test_all = np.hstack((li2, X_test[['Business Class', 'Economy', 'First Class', 'Premium Economy', 'seat_comfort_rating_1', 'cabin_staff_rating_1', 'food_beverages_rating_1']]))
print("the overall dimension of matrix is")
X_test_all.shape

the overall dimension of matrix is


(600, 307)

In [197]:
from sklearn.neighbors import KNeighborsClassifier
classifier = RakelO(
    base_classifier=KNeighborsClassifier(n_neighbors = 1),
    base_classifier_require_dense=[False, False],
    labelset_size=2,
    model_count=5
)



In [40]:
len(X_tr_all)

1400

In [198]:
classifier.fit(X_tr_all, y_tr)
#prediction = classifier.predict(X_test_all, y_test)

MajorityVotingClassifier(classifier=LabelPowerset(classifier=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform'),
       require_dense=[False, False]),
             clusterer=RandomLabelSpaceClusterer(allow_overlap=True, cluster_count=5, cluster_size=2),
             require_dense=[False, False])

In [199]:
prediction = classifier.predict(X_test_all)

In [200]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, prediction)

0.6716666666666666

In [201]:
from sklearn.metrics import hamming_loss
hamming_loss(y_test, prediction, labels=None, sample_weight=None)

0.18055555555555555

In [202]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test, prediction, sample_weight=None)

0.7422222222222223

In [203]:
from sklearn.metrics import f1_score
f1_score(y_test, prediction,  labels=None, pos_label=1, average='micro', sample_weight=None)

0.8421563865954348

In [204]:
from sklearn.metrics import f1_score
f1_score(y_test, prediction,  labels=None, pos_label=1, average='macro', sample_weight=None)

0.8415946756541718

In [205]:
from sklearn import svm

In [206]:
classifier = RakelO(
    base_classifier=svm.SVC(gamma='scale'),
    base_classifier_require_dense=[False, False],
    labelset_size=2,
    model_count=5
)


In [207]:
classifier.fit(X_tr_all, y_tr.values)

MajorityVotingClassifier(classifier=LabelPowerset(classifier=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       require_dense=[False, False]),
             clusterer=RandomLabelSpaceClusterer(allow_overlap=True, cluster_count=5, cluster_size=2),
             require_dense=[False, False])

In [208]:
prediction = classifier.predict(X_test_all)

In [209]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, prediction)

0.7916666666666666

In [53]:
from sklearn.metrics import hamming_loss
hamming_loss(y_test, prediction, labels=None, sample_weight=None)

0.12444444444444444

In [54]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test, prediction, sample_weight=None)

0.8344444444444444

In [55]:
from sklearn.metrics import f1_score
f1_score(y_test, prediction,  labels=None, pos_label=1, average='micro', sample_weight=None)

0.8914728682170542

In [56]:
from sklearn.metrics import f1_score
f1_score(y_test, prediction,  labels=None, pos_label=1, average='macro', sample_weight=None)

0.8912287552911161

In [57]:
from sklearn.neural_network import MLPClassifier

In [58]:
classifier = RakelO(
    base_classifier=MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1),
    base_classifier_require_dense=[False, False],
    labelset_size=2,
    model_count=5
)

In [59]:
classifier.fit(X_tr_all, y_tr)

MajorityVotingClassifier(classifier=LabelPowerset(classifier=MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n...     validation_fraction=0.1, verbose=False, warm_start=False),
       require_dense=[False, False]),
             clusterer=RandomLabelSpaceClusterer(allow_overlap=True, cluster_count=5, cluster_size=2),
             require_dense=[False, False])

In [60]:
prediction = classifier.predict(X_test_all)

In [61]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, prediction)

0.755

In [62]:
from sklearn.metrics import hamming_loss
hamming_loss(y_test, prediction, labels=None, sample_weight=None)

0.1322222222222222

In [63]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test, prediction, sample_weight=None)

0.8136111111111111

In [64]:
from sklearn.metrics import f1_score
f1_score(y_test, prediction,  labels=None, pos_label=1, average='micro', sample_weight=None)

0.8801611278952668

In [65]:
from sklearn.metrics import f1_score
f1_score(y_test, prediction,  labels=None, pos_label=1, average='macro', sample_weight=None)

0.8794090642673874

In [66]:
from sklearn.linear_model import LogisticRegression

In [67]:
classifier = RakelO(
    base_classifier=LogisticRegression(),
    base_classifier_require_dense=[False, False],
    labelset_size=2,
    model_count=5
)

In [68]:
classifier.fit(X_tr_all, y_tr)

C:\Users\ABHI\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ABHI\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


MajorityVotingClassifier(classifier=LabelPowerset(classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       require_dense=[False, False]),
             clusterer=RandomLabelSpaceClusterer(allow_overlap=True, cluster_count=5, cluster_size=2),
             require_dense=[False, False])

In [69]:
prediction = classifier.predict(X_test_all)

In [70]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, prediction)

0.795

In [71]:
from sklearn.metrics import hamming_loss
hamming_loss(y_test, prediction, labels=None, sample_weight=None)

0.12166666666666667

In [72]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test, prediction, sample_weight=None)

0.8366666666666667

In [73]:
from sklearn.metrics import f1_score
f1_score(y_test, prediction,  labels=None, pos_label=1, average='micro', sample_weight=None)

0.8929095354523228

In [74]:
from sklearn.metrics import f1_score
f1_score(y_test, prediction,  labels=None, pos_label=1, average='macro', sample_weight=None)

0.892645278008783

In [75]:
from sklearn.ensemble import RandomForestClassifier


In [76]:
classifier = RakelO(
    base_classifier=RandomForestClassifier(),
    base_classifier_require_dense=[False, False],
    labelset_size=2,
    model_count=5
)

In [77]:
classifier.fit(X_tr_all, y_tr)

C:\Users\ABHI\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\ABHI\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\ABHI\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\ABHI\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\ABHI\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

MajorityVotingClassifier(classifier=LabelPowerset(classifier=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
 ...e, random_state=None, verbose=0,
            warm_start=False),
       require_dense=[False, False]),
             clusterer=RandomLabelSpaceClusterer(allow_overlap=True, cluster_count=5, cluster_size=2),
             require_dense=[False, False])

In [78]:
prediction = classifier.predict(X_test_all)

In [79]:
accuracy_score(y_test, prediction)

0.7466666666666667

In [80]:
hamming_loss(y_test, prediction, labels=None, sample_weight=None)

0.13555555555555557

In [81]:
jaccard_similarity_score(y_test, prediction, sample_weight=None)

0.81

In [82]:
f1_score(y_test, prediction,  labels=None, pos_label=1, average='micro', sample_weight=None)

0.8829174664107485

In [83]:
f1_score(y_test, prediction,  labels=None, pos_label=1, average='macro', sample_weight=None)

0.882494363052975

In [84]:
clf1 = KNeighborsClassifier(n_neighbors = 1)
clf2 = RandomForestClassifier()
clf3 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf4 = LogisticRegression()
clf5 = svm.SVC(gamma='scale')

In [85]:
for clf, label in zip([clf1, clf2, clf3, clf4, clf5],['KNN', 'SVM', 'MLP', 'LP', 'RF']):
    classifier = RakelO(
        base_classifier=(clf),
        base_classifier_require_dense=[False, False],
        labelset_size=2,
        model_count=5
        )

In [86]:
classifier.fit(X_tr_all, y_tr)

MajorityVotingClassifier(classifier=LabelPowerset(classifier=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       require_dense=[False, False]),
             clusterer=RandomLabelSpaceClusterer(allow_overlap=True, cluster_count=5, cluster_size=2),
             require_dense=[False, False])

In [87]:
prediction = classifier.predict(X_test_all)

In [88]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, prediction)

0.7916666666666666

In [89]:
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.cluster import LabelCooccurrenceGraphBuilder 
from skmultilearn.ensemble import LabelSpacePartitioningClassifier

In [90]:
from skmultilearn.cluster import NetworkXLabelGraphClusterer

In [91]:
base_classifier = KNeighborsClassifier(n_neighbors = 1)

In [92]:
graph_builder = LabelCooccurrenceGraphBuilder(
    weighted = True,
    include_self_edges = False
)

In [93]:
problem_transform_classifier = LabelPowerset(classifier=base_classifier,
    require_dense=[False, False])

In [94]:
clusterer = NetworkXLabelGraphClusterer(graph_builder=graph_builder, method='louvain')

In [95]:
classifier = LabelSpacePartitioningClassifier(problem_transform_classifier, clusterer)

In [96]:
classifier.fit(X_tr_all, y_tr)

LabelSpacePartitioningClassifier(classifier=LabelPowerset(classifier=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform'),
       require_dense=[False, False]),
                 clusterer=<skmultilearn.cluster.networkx.NetworkXLabelGraphClusterer object at 0x000000C31DF35128>,
                 require_dense=[False, False])

In [97]:
prediction = classifier.predict(X_test_all)

In [98]:
accuracy_score(y_test, prediction)

0.6716666666666666

In [99]:
hamming_loss(y_test, prediction, labels=None, sample_weight=None)

0.18055555555555555

In [100]:
jaccard_similarity_score(y_test, prediction, sample_weight=None)

0.7422222222222223

In [101]:
f1_score(y_test, prediction,  labels=None, pos_label=1, average='micro', sample_weight=None)

0.8421563865954348

In [102]:
f1_score(y_test, prediction,  labels=None, pos_label=1, average='macro', sample_weight=None)

0.8415946756541718

In [103]:
base_classifier = LogisticRegression()

In [104]:
graph_builder = LabelCooccurrenceGraphBuilder(
    weighted = True,
    include_self_edges = False
)

In [105]:
problem_transform_classifier = LabelPowerset(classifier=base_classifier,
    require_dense=[False, False])

In [106]:
clusterer = NetworkXLabelGraphClusterer(graph_builder=graph_builder, method='louvain')

In [107]:
classifier = LabelSpacePartitioningClassifier(problem_transform_classifier, clusterer)

In [108]:
classifier.fit(X_tr_all, y_tr)

C:\Users\ABHI\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ABHI\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LabelSpacePartitioningClassifier(classifier=LabelPowerset(classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       require_dense=[False, False]),
                 clusterer=<skmultilearn.cluster.networkx.NetworkXLabelGraphClusterer object at 0x000000C31DF54080>,
                 require_dense=[False, False])

In [109]:
prediction = classifier.predict(X_test_all)

In [110]:
accuracy_score(y_test, prediction)

0.795

In [111]:
hamming_loss(y_test, prediction, labels=None, sample_weight=None)

0.12055555555555555

In [112]:
jaccard_similarity_score(y_test, prediction, sample_weight=None)

0.8372222222222222

In [113]:
f1_score(y_test, prediction,  labels=None, pos_label=1, average='micro', sample_weight=None)

0.8940946803318692

In [114]:
f1_score(y_test, prediction,  labels=None, pos_label=1, average='macro', sample_weight=None)

0.8938462217239196

In [115]:
base_classifier = svm.SVC(gamma = "scale")

In [116]:
graph_builder = LabelCooccurrenceGraphBuilder(
    weighted = True,
    include_self_edges = False
)

In [117]:
problem_transform_classifier = LabelPowerset(classifier=base_classifier,
    require_dense=[False, False])

In [118]:
clusterer = NetworkXLabelGraphClusterer(graph_builder=graph_builder, method='louvain')

In [119]:
classifier = LabelSpacePartitioningClassifier(problem_transform_classifier, clusterer)

In [120]:
classifier.fit(X_tr_all, y_tr)

LabelSpacePartitioningClassifier(classifier=LabelPowerset(classifier=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       require_dense=[False, False]),
                 clusterer=<skmultilearn.cluster.networkx.NetworkXLabelGraphClusterer object at 0x000000C31DF3B550>,
                 require_dense=[False, False])

In [121]:
prediction = classifier.predict(X_test_all)

In [122]:
accuracy_score(y_test, prediction)

0.79

In [123]:
hamming_loss(y_test, prediction, labels=None, sample_weight=None)

0.12611111111111112

In [124]:
jaccard_similarity_score(y_test, prediction, sample_weight=None)

0.8327777777777777

In [125]:
f1_score(y_test, prediction,  labels=None, pos_label=1, average='micro', sample_weight=None)

0.8901790033865505

In [126]:
f1_score(y_test, prediction,  labels=None, pos_label=1, average='macro', sample_weight=None)

0.8899353592516771

In [127]:
base_classifier=MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

In [128]:
graph_builder = LabelCooccurrenceGraphBuilder(
    weighted = True,
    include_self_edges = False
)

In [129]:
problem_transform_classifier = LabelPowerset(classifier=base_classifier,
    require_dense=[False, False])

In [130]:
clusterer = NetworkXLabelGraphClusterer(graph_builder=graph_builder, method='louvain')

In [131]:
classifier = LabelSpacePartitioningClassifier(problem_transform_classifier, clusterer)

In [132]:
classifier.fit(X_tr_all, y_tr)

LabelSpacePartitioningClassifier(classifier=LabelPowerset(classifier=MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n...     validation_fraction=0.1, verbose=False, warm_start=False),
       require_dense=[False, False]),
                 clusterer=<skmultilearn.cluster.networkx.NetworkXLabelGraphClusterer object at 0x000000C31DF5CBA8>,
                 require_dense=[False, False])

In [133]:
prediction = classifier.predict(X_test_all)

In [134]:
accuracy_score(y_test, prediction)

0.77

In [135]:
hamming_loss(y_test, prediction, labels=None, sample_weight=None)

0.1361111111111111

In [136]:
jaccard_similarity_score(y_test, prediction, sample_weight=None)

0.8225

In [137]:
f1_score(y_test, prediction,  labels=None, pos_label=1, average='micro', sample_weight=None)

0.8810101991257893

In [138]:
f1_score(y_test, prediction,  labels=None, pos_label=1, average='macro', sample_weight=None)

0.8806658586898974

In [139]:
base_classifier=RandomForestClassifier()

In [140]:
graph_builder = LabelCooccurrenceGraphBuilder(
    weighted = True,
    include_self_edges = False
)


In [141]:
problem_transform_classifier = LabelPowerset(classifier=base_classifier,
    require_dense=[False, False])

In [142]:
clusterer = NetworkXLabelGraphClusterer(graph_builder=graph_builder, method='louvain')

In [143]:
classifier = LabelSpacePartitioningClassifier(problem_transform_classifier, clusterer)

In [144]:
classifier.fit(X_tr_all, y_tr)

C:\Users\ABHI\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


LabelSpacePartitioningClassifier(classifier=LabelPowerset(classifier=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
 ...e, random_state=None, verbose=0,
            warm_start=False),
       require_dense=[False, False]),
                 clusterer=<skmultilearn.cluster.networkx.NetworkXLabelGraphClusterer object at 0x000000C31DF06A90>,
                 require_dense=[False, False])

In [145]:
prediction = classifier.predict(X_test_all)

In [146]:
accuracy_score(y_test, prediction)

0.755

In [147]:
hamming_loss(y_test, prediction, labels=None, sample_weight=None)

0.15888888888888889

In [148]:
jaccard_similarity_score(y_test, prediction, sample_weight=None)

0.8066666666666665

In [149]:
f1_score(y_test, prediction,  labels=None, pos_label=1, average='micro', sample_weight=None)

0.8634192932187201

In [150]:
f1_score(y_test, prediction,  labels=None, pos_label=1, average='macro', sample_weight=None)

0.8630836403609393

In [210]:
clf1 = KNeighborsClassifier(n_neighbors = 1)
clf2 = RandomForestClassifier()
clf3 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf4 = LogisticRegression()
clf5 = svm.SVC(gamma='scale')

In [211]:
for clf, label in zip([clf1, clf2, clf3, clf4, clf5],['KNN', 'SVM', 'MLP', 'LP', 'RF']):
    base_classifier = clf
    graph_builder = LabelCooccurrenceGraphBuilder(
    weighted = True,
    include_self_edges = False)
    problem_transform_classifier = LabelPowerset(classifier=base_classifier,
    require_dense=[False, False])
    clusterer = NetworkXLabelGraphClusterer(graph_builder=graph_builder, method='louvain')
    classifier = LabelSpacePartitioningClassifier(problem_transform_classifier, clusterer)
    classifier.fit(X_tr_all, y_tr)

C:\Users\ABHI\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\ABHI\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ABHI\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [153]:
prediction = classifier.predict(X_test_all)

In [154]:
accuracy_score(y_test, prediction)

0.79

In [155]:
hamming_loss(y_test, prediction, labels=None, sample_weight=None)

0.12611111111111112

In [156]:
jaccard_similarity_score(y_test, prediction, sample_weight=None)

0.8327777777777777

In [157]:
f1_score(y_test, prediction,  labels=None, pos_label=1, average='micro', sample_weight=None)

0.8901790033865505

In [158]:
f1_score(y_test, prediction,  labels=None, pos_label=1, average='macro', sample_weight=None)

0.8899353592516771